In [5]:
import json
import sys

import mlflow
import numpy as np
import pandas as pd

from tqdm import tqdm

sys.path.append('../src/')

from modules.config import Config

# Variables

In [6]:
MODEL_DIR = "../models/31f693930f4646c0a2767a9aea2b037d/artifacts/model"
CONFIG_DIR = "../configs/config_v7.json"


with open(CONFIG_DIR, "r") as file:
    config = Config(**json.load(file))

model = mlflow.sklearn.load_model(MODEL_DIR)

c:\Users\FLP-13-Rimba\miniconda3\envs\project-mix-design\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
a1 = 0.6333
a2 = 0.2605
a3 = 0.1062

CS = 29.18
PC = 932_450.0
CE = 350.155

CONSTRAINTS = {
    "water": (164.0, 205.0),
    "cement": (327.0, 440.0),
    "fine_aggregate": (715.0, 764.0),
    "coarse_aggregate": (1072.0, 1146.0),
    "admixture": (0.0, 8.16),
}

DENSITY = {
    "water": 1000,
    "cement": 3100,
    "fine_aggregate": 2687,
    "coarse_aggregate": 2630,
    "admixture": 1170,
}

COST = {
    "water": 10,
    "cement": 1300,
    "fine_aggregate": 150,
    "coarse_aggregate": 275,
    "admixture": 45000,
}

EFFICIENCY = {
    "water": 0.000344,
    "cement": 0.912000,
    "fine_aggregate": 0.007470,
    "coarse_aggregate": 0.007470,
    "admixture": 1.80000,
}

# Functions

## Model Prediction

In [8]:
def predict(
    water,
    cement,
    fine_aggregate,
    coarse_aggregate,
    admixture,
    age_days=28,
    diameter=150.0,
    height=300.0,
):
    fas = water / cement
    area = np.pi * (diameter / 2) ** 2

    data = {
        "age_days": age_days,
        "cement": cement,
        "water": water,
        "fas": fas,
        "fine_aggregate_kg": fine_aggregate,
        "coarse_aggregate_kg": coarse_aggregate,
        "sikacim_kg": admixture,
        "fas_kg": fas,
        "diameter": diameter,
        "height": height,
    }

    data = pd.Series(data).to_frame(name=0).T
    pred_kN = model.predict(data[config.features])[0]
    pred_MPa = 1000 * pred_kN / area

    return pred_MPa

In [9]:
predict(water=205, cement=408, fine_aggregate=715, coarse_aggregate=1072, admixture=0.0)

35.75834474476819

## Calculation

In [10]:
def calculate_cost(water, cement, fine_aggregate, coarse_aggregate, admixture):
    cost = 0
    cost += water * COST["water"]
    cost += cement * COST["cement"]
    cost += fine_aggregate * COST["fine_aggregate"]
    cost += coarse_aggregate * COST["coarse_aggregate"]
    cost += admixture * COST["admixture"]
    return cost


def calculate_efficiency(water, cement, fine_aggregate, coarse_aggregate, admixture):
    efficiency = 0
    efficiency += water * EFFICIENCY["water"]
    efficiency += cement * EFFICIENCY["cement"]
    efficiency += fine_aggregate * EFFICIENCY["fine_aggregate"]
    efficiency += coarse_aggregate * EFFICIENCY["coarse_aggregate"]
    efficiency += admixture * EFFICIENCY["admixture"]
    return efficiency


def is_fulfilling_constraint(water, cement, fine_aggregate, coarse_aggregate, admixture, return_each=False):
    sum_mass_per_density = 0
    is_within_bounds = True

    is_within_bounds &= CONSTRAINTS["water"][0] <= water <= CONSTRAINTS["water"][1]
    is_within_bounds &= CONSTRAINTS["cement"][0] <= cement <= CONSTRAINTS["cement"][1]
    is_within_bounds &= CONSTRAINTS["fine_aggregate"][0] <= fine_aggregate <= CONSTRAINTS["fine_aggregate"][1]
    is_within_bounds &= CONSTRAINTS["coarse_aggregate"][0] <= coarse_aggregate <= CONSTRAINTS["coarse_aggregate"][1]
    is_within_bounds &= CONSTRAINTS["admixture"][0] <= admixture <= CONSTRAINTS["admixture"][1]

    sum_mass_per_density += water / DENSITY["water"]
    sum_mass_per_density += cement / DENSITY["cement"]
    sum_mass_per_density += fine_aggregate / DENSITY["fine_aggregate"]
    sum_mass_per_density += coarse_aggregate / DENSITY["coarse_aggregate"]
    sum_mass_per_density += admixture / DENSITY["admixture"]

    if return_each:
        return is_within_bounds, sum_mass_per_density

    return (is_within_bounds & (1.0 <= sum_mass_per_density <= 1.02))

In [8]:
print(
    calculate_cost(
        water=205, cement=408, fine_aggregate=715, coarse_aggregate=1072, admixture=0.0
    )
)

print(
    calculate_efficiency(
        water=205, cement=408, fine_aggregate=715, coarse_aggregate=1072, admixture=0.0
    )
)

print(
    is_fulfilling_constraint(
        water=205, cement=408, fine_aggregate=715, coarse_aggregate=1072, admixture=0.0
    )
)

934500.0
385.51541
True


## Objective Function

In [20]:
class Objective:
    def __init__(self, a1, a2, a3, CS, PC, CE, age_days=28, diameter=150.0, height=300.0):
        self.a1 = a1
        self.a2 = a2
        self.a3 = a3
        self.CS = CS
        self.PC = PC
        self.CE = CE
        self.age_days = age_days
        self.diameter = diameter
        self.height = height

    def __call__(self, water, cement, fine_aggregate, coarse_aggregate, admixture, return_each=False):
        if not is_fulfilling_constraint(water, cement, fine_aggregate, coarse_aggregate, admixture):
            return 1e6

        cost = calculate_cost(water, cement, fine_aggregate, coarse_aggregate, admixture)
        efficiency = calculate_efficiency(water, cement, fine_aggregate, coarse_aggregate, admixture)
        pred = predict(
            water, cement, fine_aggregate, coarse_aggregate, admixture,
            age_days=self.age_days, diameter=self.diameter, height=self.height
        )
        
        error_cs = abs(self.CS - pred) / self.CS
        error_pc = abs(self.PC - cost) / self.PC
        error_ce = abs(self.CE - efficiency) / self.CE

        if return_each:
            return error_cs, error_pc, error_ce

        return self.a1 * error_cs + self.a2 * error_pc + self.a3 * error_ce

        # return (
        #     self.a1 * (self.CS - pred) ** 2
        #     + self.a2 * ((self.PC / 1000.) - (cost / 1000.)) ** 2
        #     + self.a3 * (self.CE - efficiency) ** 2
        # )

class ObjectiveWithSlump(Objective):
    def __init__(self, *args, target_slump=(8, 12), slump_func, **kwargs):
        super().__init__(*args, **kwargs)
        self.target_slump = target_slump
        self.slump_func = slump_func
    
    def error_slump(self, slump):
        if self.target_slump[0] <= slump <= self.target_slump[1]:
            return 0

        mid = (self.target_slump[0] + self.target_slump[1]) / 2
        return abs(mid - slump) / mid

    def __call__(self, water, cement, fine_aggregate, coarse_aggregate, admixture, return_each=False):
        slump = self.slump_func(water, cement, fine_aggregate, coarse_aggregate, admixture)

        if return_each:
            error_cs, error_pc, error_ce = super().__call__(
                water, cement, fine_aggregate, coarse_aggregate, admixture, return_each=True
            )
            error_slump = self.error_slump(slump)
            return error_cs, error_pc, error_ce, error_slump
        else:
            error = super().__call__(
                water, cement, fine_aggregate, coarse_aggregate, admixture
            )
            error += self.error_slump(slump)
            return error

In [12]:
objective = Objective(a1=0.6333, a2=0.2605, a3=0.1062, CS=29.18, PC=932_450.0, CE=350.155)

objective(water=205, cement=408, fine_aggregate=715, coarse_aggregate=1072, admixture=0.0)

0.15406859323451014

## Particle Swarm Optimization

In [13]:
def particle_swarm_optimization(
    objective,
    n_dimensions=5,
    n_particles=100,
    c1=2.0,
    c2=2.0,
    Gk=50,
    w=0.9,
    verbose=True
):
    # Initialize the particles within the constraints defined in `CONSTRAINTS`
    particles = np.random.uniform(
        low=[v[0] for v in CONSTRAINTS.values()],
        high=[v[1] for v in CONSTRAINTS.values()],
        size=(n_particles, n_dimensions),
    )

    # Initialize the velocities
    velocities = np.zeros((n_particles, n_dimensions))

    # Initialize the personal best positions and values
    personal_best_positions = particles.copy()
    personal_best_values = np.array([objective(*p) for p in particles])

    # Initialize the global best position and value
    global_best_position = particles[np.argmin(personal_best_values)]
    global_best_value = np.min(personal_best_values)
    
    if verbose:
        pbar = tqdm(range(Gk))
    else:
        pbar = range(Gk)

    for k in pbar:
        for i in range(n_particles):
            # Update the velocities
            velocities[i] = (
                w * velocities[i]
                + c1 * np.random.rand() * (personal_best_positions[i] - particles[i])
                + c2 * np.random.rand() * (global_best_position - particles[i])
            )

            # Update the particles
            particles[i] += velocities[i]

            # Update the personal best positions and values
            value = objective(*particles[i])
            if value < personal_best_values[i]:
                personal_best_values[i] = value
                personal_best_positions[i] = particles[i]

            # Update the global best position and value
            if value < global_best_value:
                global_best_value = value
                global_best_position = particles[i]

    return global_best_position, global_best_value

# Running

In [24]:
def show_result(global_best_position, objective):
    water, cement, fine_aggregate, coarse_aggregate, admixture = global_best_position.round(2)

    pc_star = int(calculate_cost(water, cement, fine_aggregate, coarse_aggregate, admixture))
    ce_star = calculate_efficiency(water, cement, fine_aggregate, coarse_aggregate, admixture)
    cs_star = predict(
        water, cement, fine_aggregate, coarse_aggregate, admixture, age_days=objective.age_days
    )
    slump_star = objective.slump_func(
        water, cement, fine_aggregate, coarse_aggregate, admixture
    )
    is_within_constraint, total_density = is_fulfilling_constraint(
        water, cement, fine_aggregate, coarse_aggregate, admixture, return_each=True
    )

    print(f'Compressive strength (CS*): {cs_star:.2f}')
    print(f'Cost (PC*): {pc_star:,}')
    print(f'Carbon emission (CE*): {ce_star:.2f}')
    print(f'Slump (S*): {slump_star:.2f}')

    print(f'Is within constraint: {is_within_constraint}')
    print(f'Total density: {total_density:.4f}')

    print('Best solution:', json.dumps({
        'water': water,
        'cement': cement,
        'fine_aggregate': fine_aggregate,
        'coarse_aggregate': coarse_aggregate,
        'admixture': admixture,
    }, indent=4))

def search_best_seed(objective, n_seeds=50, **pso_kwargs):
    best_seed, best_value = None, 1e6

    for seed in tqdm(range(n_seeds)):
        np.random.seed(seed)
        global_best_position_1, global_best_value = particle_swarm_optimization(
            objective, verbose=False, **pso_kwargs
        )
        
        if global_best_value < best_value and is_fulfilling_constraint(*global_best_position_1):
            best_seed, best_value = seed, global_best_value
            print(f'Found new best solution with seed {seed} and value {global_best_value:.4f}')

    return best_seed, best_value

In [28]:
def slump_func(water, cement, fine_aggregate, coarse_aggregate, admixture):
    fas = water / cement
    kg_sikacim = admixture
    
    if kg_sikacim < -1:
        return 999_999

    slump = (
        57.34 * fas +
        0.26 * water +
        0.96 * kg_sikacim +
        3.79 * np.log1p(kg_sikacim) +
        -73.48
    )

    return slump

## Scenario With Slump 1

In [26]:
objective_1 = ObjectiveWithSlump(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=30, PC=932_450.0, CE=350.155, age_days=28,
    target_slump=(8, 12), slump_func=slump_func
)

best_seed, best_value = search_best_seed(objective_1, n_seeds=20, n_particles=200, Gk=50)

np.random.seed(best_seed)
global_best_position_1, global_best_value_1 = particle_swarm_optimization(objective_1, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_1)
print('Best objective:', objective_1(*global_best_position_1))
print('CS, PC, CE, slump % error:', objective_1(*global_best_position_1, return_each=True))
print()

show_result(global_best_position_1, objective_1)

 50%|█████     | 10/20 [00:22<00:22,  2.28s/it]

Found new best solution with seed 9 and value 0.0092


100%|██████████| 50/50 [00:01<00:00, 25.60it/s]


PSO global best value: 0.009169286688636221
Best objective: 0.6696682617051104
CS, PC, CE % error: (0.3156809532073782, 0.007321518192821105, 0.011383310173673415, 0.46663135100920383)

Compressive strength (CS*): 39.47
Cost (PC*): 925,555
Carbon emission (CE*): 354.14
Slump (S*): 5.33
Is within constraint: True
Total density: 1.0098
Best solution: {
    "water": 186.32,
    "cement": 372.18,
    "fine_aggregate": 726.99,
    "coarse_aggregate": 1137.49,
    "admixture": 0.4
}


## Scenario With Slump 2

In [29]:
objective_2 = ObjectiveWithSlump(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=28.18, PC=932_450.0, CE=350.155, age_days=3,
    target_slump=(8, 12), slump_func=slump_func
)

best_seed, best_value = search_best_seed(objective_2, n_seeds=20, n_particles=200, Gk=50)

np.random.seed(best_seed)
global_best_position_2, global_best_value_2 = particle_swarm_optimization(objective_2, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_2)
print('Best objective:', objective_2(*global_best_position_2))
print('CS, PC, CE, slump % error:', objective_1(*global_best_position_1, return_each=True))
print()

show_result(global_best_position_2, objective_2)

 25%|██▌       | 5/20 [00:12<00:34,  2.32s/it]

Found new best solution with seed 4 and value 0.0282


 90%|█████████ | 18/20 [00:58<00:07,  3.93s/it]

Found new best solution with seed 17 and value 0.0232


100%|██████████| 50/50 [00:02<00:00, 18.46it/s]


PSO global best value: 0.023195923401941904
Best objective: 0.09269693143627379
CS, PC, CE, slump % error: (0.3156809532073782, 0.007321518192821105, 0.011383310173673415, 0.46663135100920383)

Compressive strength (CS*): 24.51
Cost (PC*): 949,441
Carbon emission (CE*): 367.88
Slump (S*): 8.08
Is within constraint: True
Total density: 1.0108
Best solution: {
    "water": 193.14,
    "cement": 386.86,
    "fine_aggregate": 738.77,
    "coarse_aggregate": 1097.55,
    "admixture": 0.71
}


## Scenario With Slump 3

In [31]:
objective_3 = ObjectiveWithSlump(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=21.91, PC=932_450.0, CE=350.155, age_days=3,
    target_slump=(8, 12), slump_func=slump_func
)

best_seed, best_value = search_best_seed(objective_3, n_seeds=12, n_particles=200, Gk=50)

np.random.seed(best_seed)
global_best_position_3, global_best_value_3 = particle_swarm_optimization(objective_3, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_3)
print('Best objective:', objective_3(*global_best_position_3))
print('CS, PC, CE, slump % error:', objective_1(*global_best_position_1, return_each=True))
print()

show_result(global_best_position_3, objective_3)

 17%|█▋        | 2/12 [00:03<00:19,  1.99s/it]

Found new best solution with seed 1 and value 0.0125


100%|██████████| 50/50 [00:03<00:00, 15.43it/s]


PSO global best value: 0.012473184202106087
Best objective: 0.1792406417459696
CS, PC, CE, slump % error: (0.3156809532073782, 0.007321518192821105, 0.011383310173673415, 0.46663135100920383)

Compressive strength (CS*): 27.56
Cost (PC*): 973,117
Carbon emission (CE*): 365.71
Slump (S*): 8.12
Is within constraint: True
Total density: 1.0099
Best solution: {
    "water": 188.66,
    "cement": 383.29,
    "fine_aggregate": 737.76,
    "coarse_aggregate": 1109.6,
    "admixture": 1.27
}


## Scenario 1

In [64]:
objective_1 = Objective(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=29.18, PC=932_450.0, CE=350.155, age_days=28
)

best_seed, best_value = search_best_seed(objective_1, n_seeds=20, n_particles=200, Gk=50)

 15%|█▌        | 3/20 [00:07<00:47,  2.79s/it]

Found new best solution with seed 2 and value 0.0006


100%|██████████| 20/20 [01:11<00:00,  3.59s/it]

Found new best solution with seed 19 and value 0.0005


In [71]:
objective_2 = Objective(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=29.18, PC=932_450.0, CE=350.155, age_days=3
)

best_seed, best_value = search_best_seed(objective_2, n_seeds=12, n_particles=200, Gk=50)

np.random.seed(best_seed)
global_best_position_1, global_best_value_1 = particle_swarm_optimization(objective_1, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_1)
print('Best objective:', objective_1(*global_best_position_1))
print('CS, PC, CE % error:', objective_1(*global_best_position_1, return_each=True))
print()

show_result(global_best_position_1, objective_1)

100%|██████████| 50/50 [00:04<00:00, 10.61it/s]


PSO global best value: 0.0005409535288790171
Best objective: 0.22888188336127388
CS, PC, CE % error: (0.3566529899383197, 0.0018676752234365187, 0.0237948722940758)

Compressive strength (CS*): 39.59
Cost (PC*): 930,824
Carbon emission (CE*): 358.49
Is within constraint: True
Total density: 1.0037
Best solution: {
    "water": 185.7,
    "cement": 376.97,
    "fine_aggregate": 718.27,
    "coarse_aggregate": 1127.33,
    "admixture": 0.47
}


## Scenario 2

In [81]:
objective_2 = Objective(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=29.18, PC=932_450.0, CE=350.155, age_days=3
)

best_seed, best_value = search_best_seed(objective_2, n_seeds=12, n_particles=200, Gk=50)

 17%|█▋        | 2/12 [00:04<00:23,  2.39s/it]

Found new best solution with seed 1 and value 0.0315


100%|██████████| 12/12 [00:23<00:00,  1.92s/it]

Found new best solution with seed 11 and value 0.0309


In [82]:
np.random.seed(best_seed)
global_best_position_2, global_best_value_2 = particle_swarm_optimization(objective_2, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_2)
print('Best objective:', objective_2(*global_best_position_2))
print('CS, PC, CE % error:', objective_2(*global_best_position_2, return_each=True))
print()

show_result(global_best_position_2, objective_2)

100%|██████████| 50/50 [00:02<00:00, 19.82it/s]


PSO global best value: 0.030919435845807867
Best objective: 0.03257064063213507
CS, PC, CE % error: (0.04497522690088316, 0.00517691689344804, 0.025793244680438383)

Compressive strength (CS*): 27.87
Cost (PC*): 937,127
Carbon emission (CE*): 359.18
Is within constraint: True
Total density: 1.0072
Best solution: {
    "water": 178.77,
    "cement": 377.44,
    "fine_aggregate": 743.85,
    "coarse_aggregate": 1129.42,
    "admixture": 0.5
}


## Scenario 3

In [87]:
objective_3 = Objective(
    a1=0.6333, a2=0.2605, a3=0.1062, CS=21., PC=932_450.0, CE=350.155, age_days=3
)

best_seed, best_value = search_best_seed(objective_3, n_seeds=12, n_particles=200, Gk=50)

 25%|██▌       | 3/12 [00:11<00:36,  4.07s/it]

Found new best solution with seed 2 and value 0.0006


100%|██████████| 12/12 [00:41<00:00,  3.48s/it]


In [88]:
np.random.seed(best_seed)
global_best_position_3, global_best_value_3 = particle_swarm_optimization(objective_3, Gk=50, n_particles=200)

print()
print('PSO global best value:', global_best_value_3)
print('Best objective:', objective_3(*global_best_position_3))
print('CS, PC, CE % error:', objective_3(*global_best_position_3, return_each=True))
print()

show_result(global_best_position_3, objective_3)

100%|██████████| 50/50 [00:05<00:00,  9.14it/s]


PSO global best value: 0.000607016664873828
Best objective: 0.0018210478501313708
CS, PC, CE % error: (0.0004601497399595324, 0.005692862229281355, 0.00043921289159327574)

Compressive strength (CS*): 21.01
Cost (PC*): 927,230
Carbon emission (CE*): 350.31
Is within constraint: True
Total density: 1.0059
Best solution: {
    "water": 193.32,
    "cement": 367.43,
    "fine_aggregate": 746.7,
    "coarse_aggregate": 1092.85,
    "admixture": 0.78
}
